# Eigengene Single-Variant Association Preparation

**Created**: 25 November 2021

## Environment

In [1]:
if (!requireNamespace("BiocManager", quietly = TRUE)) {
    install.packages("BiocManager")
}

if (!requireNamespace("ComplexHeatmap", quietly=TRUE)) {
    BiocManager::install("ComplexHeatmap")
}

In [2]:
library(tidyverse)
library(RColorBrewer)
library(ComplexHeatmap)
library(data.table)

setwd("~/eQTL_pQTL_Characterization/")

source("04_Expression/scripts/utils/ggplot_theme.R")

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: grid

ComplexHeatmap version 2.6.2
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and 

## Load Data

In [3]:
demographics <- readxl::read_xls("/nfs/team282/data/gains_team282/ClinicalData/DEMO_12jun2019.xls", sheet=1) %>%
    dplyr::mutate(sex=as.numeric(sex)) %>%
    dplyr::mutate(diagnosis=as.numeric(diagnosis)) %>%
    dplyr::mutate(SubjectBarCode=gsub("^GA", "", SubjectBarCode))

In [4]:
srs.info <- read.table("/nfs/team282/data/gains_team282/full-gains-SRS-predictions_mNN-RF.tsv", header=T) %>%
    dplyr::mutate(Sample_id=gsub("^GA", "", Sample_id)) %>%
    dplyr::filter(Assay=="RNA-seq")

rownames(srs.info) <- srs.info$Sample_id

In [5]:
covs <- read.table("~/gains_team282/eqtl/data/covs_and_peer_factors.txt") %>%
    dplyr::mutate(Sample.ID=gsub("^GA", "", rownames(.))) %>%
    dplyr::mutate(GAinS.ID=gsub("\\_.", "", Sample.ID))

In [6]:
held.out <- c("Neutrophils", "Lymphocytes", "Monocytes", paste0("PC", 1:7), "Diagnosis", "SRS1", "sex")
peer <- paste0("PEER_", 1:30)

covs <- covs %>%
    merge(., demographics, by.x="GAinS.ID", by.y="SubjectBarCode") %>%
    merge(., srs.info, by.x="Sample.ID", by.y=0) %>%
    dplyr::select(diagnosis=diagnosis, everything()) %>%
    dplyr::select(Sample.ID, any_of(held.out), any_of(peer)) %>%
    as.data.frame()
                                  
rownames(covs) <- covs$Sample.ID
covs <- covs %>%
    dplyr::select(-Sample.ID)
                                  
head(covs)

,Neutrophils,Lymphocytes,Monocytes,PC1,PC2,PC3,PC4,PC5,PC6,PC7,⋯,PEER_21,PEER_22,PEER_23,PEER_24,PEER_25,PEER_26,PEER_27,PEER_28,PEER_29,PEER_30
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
UK01050129_5,-1.7381318,1.8281826,1.2829455,0.177279994,0.051694602,0.02486030,-0.035102502,-0.00455149,-0.0603707,0.03964880,⋯,-0.001172263,-0.05533835,0.049742281,0.03725176,0.04495762,0.12696229,0.08175036,0.05295999,0.164950684,-0.10379386
UK01060123_3,-0.3845370,0.3878375,0.4548023,-0.000388133,-0.000674099,0.00810357,0.000212791,0.03396150,0.0266317,-0.01885280,⋯,-0.012723563,0.02226736,-0.027520355,-0.05782387,0.03653939,0.05959492,-0.02611259,-0.01790827,0.012047987,0.05554044
UK01070117_3,-0.7738039,0.5846032,0.9325467,0.016432300,-0.048464298,0.00813966,-0.047699600,-0.07926830,0.0263629,-0.08224740,⋯,0.040162142,0.06840651,0.000062529,0.05292596,0.01705806,0.05931079,0.08962730,0.03772458,0.008254956,-0.01570964
UK01080111_1,0.1305646,0.2888638,-0.6232047,0.161789000,0.036224801,0.00467903,-0.048611499,0.00180036,-0.0352357,0.05012510,⋯,-0.002928361,-0.03274319,0.088269033,0.07396927,-0.01906336,0.02454052,0.04243371,0.03055887,0.123383760,-0.02276623
UK01110093_1,1.0909779,-1.2488160,-0.4429439,-0.006296720,0.002821510,0.00699239,0.004351650,0.00268802,0.0166922,-0.00733983,⋯,-0.108193189,0.01590979,-0.093242317,-0.03037383,-0.07385907,-0.05968965,0.09695430,-0.03975484,0.130198911,-0.05011296
UK01110093_5,0.2302416,0.0000000,-0.3306479,-0.006296720,0.002821510,0.00699239,0.004351650,0.00268802,0.0166922,-0.00733983,⋯,-0.018054215,0.02094322,-0.008623131,0.09560122,-0.07802416,0.02365689,0.08040638,-0.02349218,0.189585552,-0.07635343


In [7]:
eigengenes <- read.csv("~/gains_team282/nikhil/expression/gene_expression/eigengenes.multiple.csv", row.names=1)

head(eigengenes)

,ME_1_1,ME_1_2,ME_1_3,ME_1_4,ME_1_5,ME_2_1,ME_2_2,ME_2_3,ME_2_4,ME_2_5,⋯,ME_105_1,ME_105_2,ME_105_3,ME_105_4,ME_105_5,ME_106_1,ME_106_2,ME_106_3,ME_106_4,ME_106_5
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
UK02270173_3,0.008640296,-0.04165815,0.027430556,-0.020437463,0.018138657,0.007926439,-0.03881513,0.03058366,-0.014329184,0.009126178,⋯,0.033258706,0.010337824,0.031572946,0.0308363822,-0.01556043,-0.003317747,0.01850385,0.001776021,0.034254655,0.002439359
UK15130120_3,-0.037073495,-0.01795641,-0.007283072,-0.026494085,0.003339612,-0.033179470,-0.01991901,-0.01688432,-0.014852805,0.049310271,⋯,0.051184001,-0.001611603,0.005624913,0.0273538940,-0.03670228,-0.012327128,0.04405671,0.013144923,0.017987682,-0.001779326
UK58000006_3,0.029670524,-0.03481715,0.028491140,-0.009483861,-0.005862293,0.021490110,-0.03480085,0.03014656,-0.003961906,-0.032946862,⋯,-0.006622840,-0.045791646,0.033491934,-0.0420056572,0.02749077,-0.004740631,0.04638229,-0.043309799,0.028730908,-0.024051766
UK47010004_3,0.011954974,0.01091971,0.038665347,0.003726632,0.052019153,0.007360515,0.01763746,0.03490663,-0.005573984,-0.016190548,⋯,-0.003072117,-0.010005728,0.019263388,0.0009626484,0.06116524,0.012014706,0.01637633,-0.003045472,-0.018223178,-0.020637855
UK42020088_5,0.009818957,0.02100233,-0.019852180,-0.013052613,-0.030223051,0.008322535,0.01712902,-0.02394729,0.003193689,0.019716419,⋯,-0.032697222,-0.001257679,-0.019760950,-0.0079000979,0.02988006,0.007915598,-0.01649680,0.013250580,-0.031445321,-0.014168970
UK47490007_3,-0.021930377,0.03807246,0.042329128,-0.025160071,0.012732846,-0.024756322,0.03899339,0.03987212,-0.030754242,0.038550679,⋯,-0.003607270,0.043079589,0.025888343,0.0413386960,0.04360340,0.024871961,-0.03349057,0.029665102,-0.009251542,-0.052908888


In [8]:
modules <- read.csv("~/gains_team282/nikhil/expression/gene_expression/modules.csv")

In [9]:
head(modules)

,Gene,Module
,<chr>,<chr>
1,ENSG00000001167,Module_1
2,ENSG00000002330,Module_1
3,ENSG00000002822,Module_1
4,ENSG00000005175,Module_1
5,ENSG00000005194,Module_1
6,ENSG00000005893,Module_1


In [10]:
gene.exp <- read.table("/lustre/scratch119/humgen/projects/gains_team282/eqtl/data/logcpm_864_20412_hla.txt")

colnames(gene.exp) <- gsub("^GA", "", colnames(gene.exp))

In [11]:
head(gene.exp)

,UK02270173_3,UK15130120_3,UK58000006_3,UK47010004_3,UK42020088_5,UK47490007_3,UK02770164_3,UK02770164_5,UK02630151_3,UK42150107_1,⋯,UK59070043_3,UK59070043_5,UK02510223_3,UK02XX0336_5,UK29090086_3,UK02XX0335_1,UK02XX0334_3,UK01210130_3,UK01210130_5,UK01380125_1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000230521,0.6644339,0.58271604,0.4046756,0.20905157,0.2211571,0.4288519,0.5133516,0.57768877,0.6990766,0.3812037,⋯,0.3823184,0.5423684,0.86762630,0.73833461,0.6461583,0.1227063,0.9621386,0.65838459,1.03561474,0.4457301
ENSG00000225864,0.9817872,0.43879936,0.6022344,0.92340489,0.2211571,1.1450686,0.1921362,0.57768877,0.1974851,1.2290495,⋯,0.6842527,0.8539227,0.91886777,0.12581620,0.5511865,0.4384007,0.5465139,0.40803066,0.46899151,0.3544115
ENSG00000227766,1.2117156,0.30682669,0.6625010,0.79963286,0.6445686,0.3130290,0.1183231,0.16750450,0.1505900,1.0533199,⋯,0.4901384,0.4340781,0.37504164,0.04316943,0.6151905,0.3406488,0.3147798,0.43511377,0.17382742,0.1157514
ENSG00000237669,0.6420763,0.69256830,0.9792012,0.69929584,0.3765156,1.0756817,0.8912050,0.60129641,0.8534668,1.0533199,⋯,1.0790156,0.6102946,1.03953331,0.97965846,0.7061683,0.7489857,0.7919934,0.85168813,0.69120049,1.0798771
ENSG00000271581,2.4744895,1.33153140,1.8310464,2.47531701,1.8812042,2.0531366,0.5972544,1.00686213,1.2811361,2.4244938,⋯,2.4728831,1.5338898,1.59080138,0.62923925,1.2804253,1.4727511,1.2187650,1.46471386,1.44139443,1.1896250
ENSG00000285647,0.0000000,0.03379072,0.0000000,0.05517067,0.6748270,0.0000000,0.0000000,0.03509293,0.6990766,1.6556266,⋯,3.4117627,2.4012963,0.04682116,0.00000000,0.0000000,0.4695662,0.3762952,0.07078527,0.07206309,0.4159282


In [12]:
lead.cis.eqtl <- read.table("/nfs/users/nfs_n/nm18/gains_team282/eqtl/cisresults/eigenMT/ciseqtl_eigenMT_corrected.txt") %>%
    dplyr::filter(Sig)

In [13]:
head(lead.cis.eqtl)

,snps,gene,statistic,pvalue,beta,se,chr,SNPpos,TSS,BF,TESTS,BF.FDR,Sig,threshold
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<lgl>,<dbl>
22,rs3131972,ENSG00000237491,5.643405,2.400875e-08,0.07174216,0.012712570,1,817341,778747,7.058573e-06,294,1.993414e-05,TRUE,9.170798e-05
23,rs3131972,ENSG00000230092,4.756587,2.329109e-06,0.06239499,0.013117600,1,817341,817712,6.917454e-04,297,1.593483e-03,TRUE,9.078164e-05
25,rs3131972,ENSG00000225880,6.292961,5.949542e-10,0.10016770,0.015917420,1,817341,827522,1.767014e-07,297,5.668498e-07,TRUE,9.078164e-05
28,rs2272757,ENSG00000188976,-7.026045,4.904859e-12,-0.05107767,0.007269761,1,946247,959309,1.545031e-09,315,5.762103e-09,TRUE,8.559411e-05
29,rs13303327,ENSG00000187961,-7.175012,1.845538e-12,-0.13519920,0.018843060,1,960326,960584,5.813445e-10,315,2.228468e-09,TRUE,8.559411e-05
30,rs13303056,ENSG00000187583,-9.081031,1.922802e-18,-0.29633770,0.032632610,1,953778,966482,6.056826e-16,315,3.262796e-15,TRUE,8.559411e-05


In [14]:
conditional.cis.eqtl <- readRDS("/nfs/users/nfs_n/nm18/gains_team282/eqtl/cisresults/conditionalanalysis/conditional_eQTL_results_final.rds")

In [15]:
head(conditional.cis.eqtl)

,SNP,Gene,eQTL_beta,eQTL_SE,pvalue,Number
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,rs10753794,ENSG00000000457,0.0498587312408011,0.00708515851579373,5.19259471106013e-12,1
2,rs10919255,ENSG00000000460,-0.0726264894633498,0.0136853304567293,1.72815915933528e-07,1
3,rs77006036,ENSG00000000460,0.166236949102582,0.0334472538699365,8.47290238610159e-07,2
4,rs12406047,ENSG00000000971,0.17135374034767,0.0379807691167313,7.51523462853891e-06,1
5,rs6696136,ENSG00000001460,0.45884434698694,0.0871081626842823,1.94887370606405e-07,1
6,rs6676449,ENSG00000001460,-0.371531778822554,0.0880672792106829,2.8471651589813e-05,2


In [16]:
ebi.studies <- fread("04_Expression/data/gwas_catalog_v1.0.2-studies_r2022-02-21.tsv", header=TRUE, quote="") %>%
    as.data.frame()

In [17]:
head(ebi.studies, n=1)

,DATE ADDED TO CATALOG,PUBMEDID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,PLATFORM [SNPS PASSING QC],ASSOCIATION COUNT,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY
,<date>,<int>,<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
1,2021-08-24,34124712,Sherva R,2021-02-28,Explor Med,www.ncbi.nlm.nih.gov/pubmed/34124712,Genome-wide association study of phenotypes measuring progression from first cocaine or opioid use to dependence reveals novel risk genes.,Cocaine dependence (time to event),"3,554 African American cases, 478 African American controls, 2,712 European ancestry cases, 915 European ancestry controls","572 African American cases, 416 African American controls, 759 European ancestry cases, 1,620 European ancestry controls",Illumina [NR] (imputed),5,cocaine dependence,http://www.ebi.ac.uk/efo/EFO_0002610,GCST012225,Genome-wide genotyping array


In [18]:
ebi.assoc <- fread("04_Expression/data/gwas_catalog_v1.0.2-associations_e105_r2022-02-21.tsv", header=TRUE, quote="") %>%
    as.data.frame()

In [19]:
head(ebi.assoc, n=1)

,DATE ADDED TO CATALOG,PUBMEDID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,⋯,PVALUE_MLOG,P-VALUE (TEXT),OR or BETA,95% CI (TEXT),PLATFORM [SNPS PASSING QC],CNV,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY
,<date>,<int>,<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2021-07-07,34127860,Robertson CC,2021-06-14,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/34127860,"Fine-mapping, trans-ancestral and genomic analyses identify causal variants, cells, genes and drug targets for type 1 diabetes.",Type 1 diabetes,"20,065 European ancestry cases, 33,065 European ancestry controls, 1,045 other-admixed ancestry cases, 1,103 other-admixed ancestry controls, 1,043 African ancestry cases, 3,206 African ancestry controls",NA,⋯,8.522879,,0.912,,Illumina [715631] (imputed),N,type 1 diabetes mellitus,http://purl.obolibrary.org/obo/MONDO_0005147,GCST90013445,Targeted genotyping array [ImmunoChip]


In [20]:
geno.bim <- fread("~/gains_team282/Genotyping/All_genotyping_merged_filtered_b38_refiltered_rsID.bim") %>%
    as.data.frame()
colnames(geno.bim) <- c("chr", "snps", "cM", "pos", "minor", "major")

In [21]:
head(geno.bim)

,chr,snps,cM,pos,minor,major
,<chr>,<chr>,<int>,<int>,<chr>,<chr>
1,1,rs3131972,0,817341,A,G
2,1,rs546843995,0,818053,0,G
3,1,rs553916047,0,818359,0,A
4,1,1:818740_T_C,0,818740,T,C
5,1,rs145604921,0,819378,0,C
6,1,rs535256652,0,821053,0,T


## Heatmap

Before performing single-variant association analysis, we want to check if any covariates are strongly associated with the module eigengenes.

In [22]:
all.vars <- merge(covs, eigengenes, by.x=0, by.y=0) %>%
    dplyr::select(-Row.names)

In [23]:
colors <- colorRampPalette(brewer.pal(11, "RdBu"))(101)

cor.mtx <- cor(all.vars)
cor.mtx.peer <- cor.mtx[peer, colnames(eigengenes)]
cor.mtx.held.out <- cor.mtx[held.out, colnames(eigengenes)]

h1 <- Heatmap(cor.mtx.peer, col=colors, heatmap_legend_param=list(
    title="Correlation",
    at=c(-1, 0, 1)
))

h2 <- Heatmap(cor.mtx.held.out, col=colors, cluster_rows=F, heatmap_legend_param=list(
    title="Correlation",
    at=c(-1, 0, 1)
))

svg("04_Expression/results/eigengene_peer_factor_correlation.svg", width=12, height=8)
h1 %v% h2
dev.off()

png 
  2

The PEER factors are built using gene expression data. It's not surprising that many of the eigengenes are associated with the PEER factors. WGCNA discarded around 8000 genes. The residual gene expression variation from these genes is likely captured by some of the PEER factors. We model this specifically using 30 gene expression PCs from the discarded genes.

The eigengenes are not associated with genotyping PCs (which is good).

![](../results/eigengene_peer_factor_correlation.svg)

Ideally, I will include only sex and genotyping PCs as covariates in this mapping, since everything else (cell counts, SRS group) are important signatures that may be correlated with module eigengenes. I will include the 30 PCs from the unassigned gene expression data to control for technical confounders.

## Save Covariates

Save list of patients (family ID in the first column and individual ID in the second column) to subset the genotyping data using PLINK. There are 638 unique patients with genotyping information.

In [24]:
eigengene.patients <- sapply(strsplit(rownames(eigengenes), "_"), function(x) { x[1] })

geno.fam <- fread("~/gains_team282/Genotyping/All_genotyping_merged_filtered_b38_refiltered_rsID.fam") %>%
    dplyr::select(Family.ID=1, Individual.ID=2) %>%
    dplyr::mutate(GAinS.ID=gsub("^GA", "", Individual.ID)) %>%
    dplyr::filter(GAinS.ID %in% eigengene.patients) %>%
    unique()

In [25]:
dim(geno.fam)

[1] 638   3

In [26]:
write.table(
    geno.fam %>% dplyr::select(Family.ID, Individual.ID), 
    "~/gains_team282/nikhil/expression/eigengene_sva/mapping_patients.txt", row.names=F, quote=F, col.names=F, sep="\t"
)

Save the list of module eigengenes as a text file. This will be used by NextFlow to parallelize the association mapping.

In [27]:
write.table(colnames(eigengenes), "~/gains_team282/nikhil/expression/eigengene_sva/mapping_eigengenes.txt", row.names=F, quote=F, col.names=F, sep="\t")

### Initial Pass

I will be using a linear mixed model (LMM) to test for association between genotypes and eigengene expression. In the past, the lab has used a likelihood test (F-Test) to compare a null model of the covariates against an alternative model where the genotype is included. I will be building the following model:

1. Let $\mathbf{E}_i\in\mathbb{R}^n$ be a vector representing the values of the $i$-th eigengene.
2. Let $\mathbf{Y}\in\mathbb{R}^{n\times c}$ be a matrix of covariates. These covariates include Sex, 16 PEER factors, and 7 Genotyping PCs.
3. Let $\mathbf{Z}\in\mathbb{R}^n$ be a vector representing the random effects. The only random effect in this model is the Patient ID.
4. Let $\mathbf{X}\in\mathbb{R}^n$ be a vector representing the genotypes of the patients.
5. Let $\beta\in\mathbb{R}$ be a scalar value representing the genotypic effect on eigengene expression.
6. Let $\mathbf{\alpha}\in\mathbb{R}^c$ be a vector of covariate effects on eigengene expression.
7. Let $\gamma\in\mathbb{R}$ be a scalar value representing the random effect on eigengene expression.

The null model is:

$$\mathbf{E}_i \sim \mathbf{Y}\alpha + \mathbf{Z}\gamma$$

The alternative model is:

$$\mathbf{E}_i \sim \mathbf{X}\beta + \mathbf{Y}\alpha + \mathbf{Z}\gamma$$

There are 823 samples from the RNA-Seq data that have genotypes as well.

In [28]:
samples.with.genotypes <- rownames(eigengenes)[
    sapply(
        strsplit(rownames(eigengenes), "_"),
        function(x) { x[1] %in% geno.fam$GAinS.ID }
    )
]

length(samples.with.genotypes)

[1] 823

In [29]:
peer.included <- peer[1:20]
peer.included

cov.names <- c("Diagnosis", "SRS1", paste0("PC", 1:7), "Neutrophils", "Lymphocytes", "Monocytes")
cov.names

rand.effect.names <- c("GAinS.ID")
rand.effect.names

[1] "PEER_1"  "PEER_2"  "PEER_3"  "PEER_4"  "PEER_5"  "PEER_6"  "PEER_7" 
 [8] "PEER_8"  "PEER_9"  "PEER_10" "PEER_11" "PEER_12" "PEER_13" "PEER_14"
[15] "PEER_15" "PEER_16" "PEER_17" "PEER_18" "PEER_19" "PEER_20"

[1] "Diagnosis"   "SRS1"        "PC1"         "PC2"         "PC3"        
 [6] "PC4"         "PC5"         "PC6"         "PC7"         "Neutrophils"
[11] "Lymphocytes" "Monocytes"

[1] "GAinS.ID"

In [30]:
mapping.data <- merge(eigengenes, covs, by=0) %>%
    dplyr::filter(Row.names %in% samples.with.genotypes) %>%
    dplyr::select(Sample.ID=Row.names, everything()) %>%
    dplyr::mutate(GAinS.ID.NonPrefix=sapply(strsplit(Sample.ID, "_"), function(x) { x[1] })) %>%
    merge(., geno.fam, by.x="GAinS.ID.NonPrefix", by.y="GAinS.ID") %>%
    dplyr::select(GAinS.ID=Individual.ID, everything()) %>%
    dplyr::select(Sample.ID, any_of(rand.effect.names), any_of(colnames(eigengenes)), any_of(cov.names), any_of(peer.included))

In [31]:
head(mapping.data)

,Sample.ID,GAinS.ID,ME_1_1,ME_1_2,ME_1_3,ME_1_4,ME_1_5,ME_2_1,ME_2_2,ME_2_3,⋯,PEER_11,PEER_12,PEER_13,PEER_14,PEER_15,PEER_16,PEER_17,PEER_18,PEER_19,PEER_20
,<I<chr>>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,UK01050129_5,UK01050129,-0.011965012,-0.09047520,-0.017610108,0.0007358012,-0.0151430087,-0.008135170,-0.09597010,-0.012628564,⋯,0.047811605,-0.016539395,0.051095940,-0.026148196,-0.021135228,0.002443923,0.032696567,0.035078023,0.03328779,-0.072080098
2,UK01060123_3,UK01060123,-0.002691514,-0.02324379,0.040388816,-0.0194287731,0.0097147859,-0.003642363,-0.02567566,0.042063568,⋯,-0.007224238,0.016796881,-0.010660484,-0.017610908,0.031190908,0.005303205,0.040227711,0.023455137,-0.06972641,-0.016429750
3,UK01070117_3,UK01070117,-0.027671656,-0.01496809,-0.008750141,-0.0280644252,-0.0200438485,-0.018676902,-0.01817045,-0.010687002,⋯,0.002017805,0.013472254,0.044990435,0.017342746,0.040239949,-0.037767619,-0.002149181,0.010321584,-0.03437537,-0.008777559
4,UK01080111_1,UK01080111,0.021899313,-0.01413116,-0.004961546,0.0193836562,-0.0677604487,0.011468622,-0.01269462,0.000280292,⋯,-0.063317396,-0.072918773,-0.010724876,-0.016138442,-0.003775158,-0.032603394,-0.068149783,-0.007441163,-0.04162392,-0.056151949
5,UK01110093_1,UK01110093,-0.053133706,0.11972216,-0.029216200,0.0042203907,0.0216303612,-0.054170247,0.11333090,-0.036116204,⋯,-0.009035902,-0.001971741,-0.003907994,-0.001863737,-0.031021271,-0.001989635,0.010791964,-0.013796184,0.09261347,-0.072555415
6,UK01110093_5,UK01110093,-0.097745041,0.08920047,0.001058428,0.0131789677,-0.0001536524,-0.093620110,0.07847404,-0.007012426,⋯,0.076265849,-0.071181759,0.045724034,-0.083036296,0.009541817,-0.043817975,0.079534695,0.008144007,0.09541912,-0.201759934


In [32]:
colnames(mapping.data)

[1] "Sample.ID"   "GAinS.ID"    "ME_1_1"      "ME_1_2"      "ME_1_3"     
  [6] "ME_1_4"      "ME_1_5"      "ME_2_1"      "ME_2_2"      "ME_2_3"     
 [11] "ME_2_4"      "ME_2_5"      "ME_3_1"      "ME_3_2"      "ME_3_3"     
 [16] "ME_3_4"      "ME_3_5"      "ME_4_1"      "ME_4_2"      "ME_4_3"     
 [21] "ME_4_4"      "ME_4_5"      "ME_5_1"      "ME_5_2"      "ME_5_3"     
 [26] "ME_5_4"      "ME_5_5"      "ME_6_1"      "ME_6_2"      "ME_6_3"     
 [31] "ME_6_4"      "ME_6_5"      "ME_7_1"      "ME_7_2"      "ME_7_3"     
 [36] "ME_7_4"      "ME_7_5"      "ME_8_1"      "ME_8_2"      "ME_8_3"     
 [41] "ME_8_4"      "ME_8_5"      "ME_9_1"      "ME_9_2"      "ME_9_3"     
 [46] "ME_9_4"      "ME_9_5"      "ME_10_1"     "ME_10_2"     "ME_10_3"    
 [51] "ME_10_4"     "ME_10_5"     "ME_11_1"     "ME_11_2"     "ME_11_3"    
 [56] "ME_11_4"     "ME_11_5"     "ME_12_1"     "ME_12_2"     "ME_12_3"    
 [61] "ME_12_4"     "ME_12_5"     "ME_13_1"     "ME_13_2"     "ME_13_3"    
 [66] "ME_13_4"     "ME_13_5"     "ME_14_1"     "ME_14_2"     "ME_14_3"    
 [71] "ME_14_4"     "ME_14_5"     "ME_15_1"     "ME_15_2"     "ME_15_3"    
 [76] "ME_15_4"     "ME_15_5"     "ME_16_1"     "ME_16_2"     "ME_16_3"    
 [81] "ME_16_4"     "ME_16_5"     "ME_17_1"     "ME_17_2"     "ME_17_3"    
 [86] "ME_17_4"     "ME_17_5"     "ME_18_1"     "ME_18_2"     "ME_18_3"    
 [91] "ME_18_4"     "ME_18_5"     "ME_19_1"     "ME_19_2"     "ME_19_3"    
 [96] "ME_19_4"     "ME_19_5"     "ME_20_1"     "ME_20_2"     "ME_20_3"    
[101] "ME_20_4"     "ME_20_5"     "ME_21_1"     "ME_21_2"     "ME_21_3"    
[106] "ME_21_4"     "ME_21_5"     "ME_22_1"     "ME_22_2"     "ME_22_3"    
[111] "ME_22_4"     "ME_22_5"     "ME_23_1"     "ME_23_2"     "ME_23_3"    
[116] "ME_23_4"     "ME_23_5"     "ME_24_1"     "ME_24_2"     "ME_24_3"    
[121] "ME_24_4"     "ME_24_5"     "ME_25_1"     "ME_25_2"     "ME_25_3"    
[126] "ME_25_4"     "ME_25_5"     "ME_26_1"     "ME_26_2"     "ME_26_3"    
[131] "ME_26_4"     "ME_26_5"     "ME_27_1"     "ME_27_2"     "ME_27_3"    
[136] "ME_27_4"     "ME_27_5"     "ME_28_1"     "ME_28_2"     "ME_28_3"    
[141] "ME_28_4"     "ME_28_5"     "ME_29_1"     "ME_29_2"     "ME_29_3"    
[146] "ME_29_4"     "ME_29_5"     "ME_30_1"     "ME_30_2"     "ME_30_3"    
[151] "ME_30_4"     "ME_30_5"     "ME_31_1"     "ME_31_2"     "ME_31_3"    
[156] "ME_31_4"     "ME_31_5"     "ME_32_1"     "ME_32_2"     "ME_32_3"    
[161] "ME_32_4"     "ME_32_5"     "ME_33_1"     "ME_33_2"     "ME_33_3"    
[166] "ME_33_4"     "ME_33_5"     "ME_34_1"     "ME_34_2"     "ME_34_3"    
[171] "ME_34_4"     "ME_34_5"     "ME_35_1"     "ME_35_2"     "ME_35_3"    
[176] "ME_35_4"     "ME_35_5"     "ME_36_1"     "ME_36_2"     "ME_36_3"    
[181] "ME_36_4"     "ME_36_5"     "ME_37_1"     "ME_37_2"     "ME_37_3"    
[186] "ME_37_4"     "ME_37_5"     "ME_38_1"     "ME_38_2"     "ME_38_3"    
[191] "ME_38_4"     "ME_38_5"     "ME_39_1"     "ME_39_2"     "ME_39_3"    
[196] "ME_39_4"     "ME_39_5"     "ME_40_1"     "ME_40_2"     "ME_40_3"    
[201] "ME_40_4"     "ME_40_5"     "ME_41_1"     "ME_41_2"     "ME_41_3"    
[206] "ME_41_4"     "ME_41_5"     "ME_42_1"     "ME_42_2"     "ME_42_3"    
[211] "ME_42_4"     "ME_42_5"     "ME_43_1"     "ME_43_2"     "ME_43_3"    
[216] "ME_43_4"     "ME_43_5"     "ME_44_1"     "ME_44_2"     "ME_44_3"    
[221] "ME_44_4"     "ME_44_5"     "ME_45_1"     "ME_45_2"     "ME_45_3"    
[226] "ME_45_4"     "ME_45_5"     "ME_46_1"     "ME_46_2"     "ME_46_3"    
[231] "ME_46_4"     "ME_46_5"     "ME_47_1"     "ME_47_2"     "ME_47_3"    
[236] "ME_47_4"     "ME_47_5"     "ME_48_1"     "ME_48_2"     "ME_48_3"    
[241] "ME_48_4"     "ME_48_5"     "ME_49_1"     "ME_49_2"     "ME_49_3"    
[246] "ME_49_4"     "ME_49_5"     "ME_50_1"     "ME_50_2"     "ME_50_3"    
[251] "ME_50_4"     "ME_50_5"     "ME_51_1"     "ME_51_2"     "ME_51_3"    
[256] "ME_51_4"     "ME_51_5"     "ME_52_1"     "ME_52_2"     "ME_52_3"    
[261] "ME_52_4"     "ME_52_5"     "ME_53_1"     "M

Save the mapping data design matrix for linear mixed modeling.

In [33]:
write.csv(mapping.data, "~/gains_team282/nikhil/expression/eigengene_sva/mapping_data.csv", row.names=F)

### Identify SNPs for mQTL Mapping

Since *trans* mapping in a genome-wide fashion will require a lot of testing (the multiple-testing burden will be high), we will use SNPs that we have a more biological hypothesis for. Specifically, I will include all lead eSNPs, all lead eSNPs from the conditional cis-eQTL analysis, and related associations from the EBI GWAS Catalog.

In [34]:
length(unique(ebi.studies[,"DISEASE/TRAIT"]))

[1] 30667

After filtering based on the genotyping in GAinS, we have 124,524 studies and 56,218 SNPs.

In [35]:
# Studies with terms specified
studies <- ebi.studies[, "STUDY ACCESSION"]

# Associations from the studies that are genome-wide significant
studies.assoc <- ebi.assoc[ebi.assoc[, "STUDY ACCESSION"] %in% studies,] %>%
    dplyr::mutate(`P-VALUE` = as.numeric(`P-VALUE`)) %>%
    dplyr::filter(`P-VALUE` < 5e-8)

# Association SNPs that overlap with genotyping data in GAinS
# Ignore SNPs that are on the X chromosome
studies.assoc <- studies.assoc %>%
    dplyr::filter(SNPS %in% geno.bim$snps[geno.bim$chr %in% as.character(1:22)])

nrow(studies.assoc)

length(unique(studies.assoc$SNPS))

[1] 124524

[1] 56218

In [36]:
mqtl.snp.table <- dplyr::bind_rows(
    lead.cis.eqtl %>%
        dplyr::mutate(source="Lead cis-eQTL SNP") %>%
        dplyr::mutate(conditional_number=NA, accession=NA) %>%
        dplyr::select(snps, source, egene=gene, conditional_number, accession),
    conditional.cis.eqtl %>%
        dplyr::mutate(source="Conditional cis-eQTL SNP") %>%
        dplyr::mutate(accession=NA) %>%
        dplyr::select(snps=SNP, source, egene=Gene, conditional_number=Number, accession),
    studies.assoc %>%
        dplyr::mutate(source="EBI GWAS Catalog") %>%
        dplyr::mutate(egene=NA, conditional_number=NA) %>%
        dplyr::select(snps=SNPS, source, egene, conditional_number, accession=37)
)

In [37]:
write.csv(mqtl.snp.table, "~/gains_team282/nikhil/expression/eigengene_sva/mqtl_snp_table.csv", quote=F, row.names=F)

In [38]:
length(unique(mqtl.snp.table$snps))

[1] 70968